In [1]:
import pandas as pd

columns = ["Delito", "Subdelito", "Fecha", "Victima", "Subvictima", "Adulto", "Genero", "Nacionalidad", "Provincia", "Canton", "Distrito"]

## Cargar los datos

In [2]:
police_15 = pd.read_csv('./Data/PJCROD_POLICIALES_V1-2015.csv',
                     encoding='latin1', names=columns, )

police_15['Fecha'] = pd.to_datetime(police_15['Fecha'])
police_15['Ano'] = police_15['Fecha'].apply(lambda x: x.year)

In [3]:
police_16 = pd.read_csv('./Data/PJCROD_POLICIALES_V1-2016.csv',
                     encoding='latin1', names=columns)

police_16['Fecha'] = pd.to_datetime(police_16['Fecha'])
police_16['Ano'] = police_16['Fecha'].apply(lambda x: x.year)

In [4]:
police_17 = pd.read_csv('./Data/PJCROD_POLICIALES_V1-2017.csv',
                     encoding='latin1', names=columns)

police_17['Fecha'] = pd.to_datetime(police_17['Fecha'])
police_17['Ano'] = police_17['Fecha'].apply(lambda x: x.year)

In [5]:
police_18 = pd.read_csv('./Data/PJCROD_POLICIALES_V1-2018.csv',
                     encoding='latin1', names=columns)

police_18['Fecha'] = pd.to_datetime(police_18['Fecha'])
police_18['Ano'] = police_18['Fecha'].apply(lambda x: x.year)

In [6]:
police_19 = pd.read_csv('./Data/PJCROD_POLICIALES_V1-2019.csv',
                     encoding='latin1', names=columns, index_col=False)

police_19['Fecha'] = pd.to_datetime(police_19['Fecha'])
police_19['Ano'] = police_19['Fecha'].apply(lambda x: x.year)

In [47]:
police = pd.concat([police_15,police_16,police_17,police_18,police_19])
police.sample(5)

,Delito,Subdelito,Fecha,Victima,Subvictima,Adulto,Genero,Nacionalidad,Provincia,Canton,Distrito,Ano
16149,ROBO DE VEHICULO,DESCUIDO,2018-04-06,VEHICULO,MICROBUS [VEHICULO],Mayor de edad ...,HOMBRE,COSTA RICA,SAN JOSE,CURRIDABAT,CURRIDABAT,2018
47031,ROBO,FORZADURA,2017-10-20,VIVIENDA,NO APLICA [VIVIENDA],Mayor de edad ...,MUJER,COSTA RICA,ALAJUELA,ATENAS,CONCEPCION,2017
32388,ASALTO,INTIMIDACION VERBAL,2015-08-06,PERSONA,PEATON [PERSONA],Mayor de edad ...,MUJER,COSTA RICA,HEREDIA,HEREDIA,MERCEDES,2015
52140,HURTO,POR DESCUIDO,2017-11-19,PERSONA,USUARIO SERVICIO PUBLICO/PRIVADO [PERSONA],Mayor de edad ...,MUJER,PAISES BAJOS (HOLANDA),PUNTARENAS,QUEPOS,QUEPOS,2017
29013,ASALTO,GOLPES,2015-07-16,PERSONA,PEATON [PERSONA],Mayor de edad ...,HOMBRE,NICARAGUA,SAN JOSE,SAN JOSE,HATILLO,2015


## Limpieza y Manipulacion de datos

In [48]:
# Cantones
police.loc[police['Canton'] == 'CAï¿½AS','Canton'] = 'CAÑAS'
police.loc[police['Canton'] == 'SARCHï¿½','Canton'] = 'SARCHI'

# Distritos
police.loc[police['Distrito'] == 'CAï¿½AS','Distrito'] = 'CAÑAS'
police.loc[police['Distrito'] == 'CAï¿½AS DULCES','Distrito'] = 'CAÑAS DULCES'
police.loc[police['Distrito'] == 'PEï¿½AS BLANCAS','Distrito'] = 'PEÑAS BLANCAS'
police.loc[police['Distrito'] == 'REVENTAZï¿½N','Distrito'] = 'REVENTAZON'

# Eliminar desconocido
desconocidos = ((police['Canton'] == 'DESCONOCIDO') |
                (police['Distrito'] == 'DESCONOCIDO') )
police.drop(police[desconocidos].index, inplace=True)

In [76]:
# Grecia --> Rio Cuarto
police.loc[police['Distrito'] == 'RIO CUARTO','Canton'] = 'RIO CUARTO'


2684     RIO CUARTO
3034     RIO CUARTO
3035     RIO CUARTO
4067     RIO CUARTO
4655     RIO CUARTO
            ...    
48893    RIO CUARTO
49052    RIO CUARTO
49352    RIO CUARTO
55290    RIO CUARTO
55963    RIO CUARTO
Name: Canton, Length: 333, dtype: object

In [50]:
police_grouped = police.groupby(['Delito', 'Subdelito', 'Ano', 'Provincia', 'Canton', 'Distrito']).size().to_frame('Count').reset_index()

In [89]:
poblaciones = pd.read_csv('./Data/Poblaciones.csv', encoding='latin1')
poblaciones.head()

,Provincia,Canton,Distrito,Poblacion
0,San Jose,San Jose,NaN,339581
1,San Jose,San Jose,Carmen,3070
2,San Jose,San Jose,Merced,14846
3,San Jose,San Jose,Hospital,23099
4,San Jose,San Jose,Catedral,15382


In [90]:
poblaciones['Provincia'] = poblaciones['Provincia'].str.upper()
poblaciones['Canton'] = poblaciones['Canton'].str.upper()
poblaciones['Distrito'] = poblaciones['Distrito'].str.upper()

In [91]:
police_final = police_grouped.merge(poblaciones, how='left',
                                    on=['Provincia','Canton', 'Distrito'])
police_final.head()


,Delito,Subdelito,Ano,Provincia,Canton,Distrito,Count,Poblacion
0,ASALTO,ARMA BLANCA,2015,ALAJUELA,ALAJUELA,ALAJUELA,114,47477.0
1,ASALTO,ARMA BLANCA,2015,ALAJUELA,ALAJUELA,CARRIZAL,2,8328.0
2,ASALTO,ARMA BLANCA,2015,ALAJUELA,ALAJUELA,DESAMPARADOS,11,31261.0
3,ASALTO,ARMA BLANCA,2015,ALAJUELA,ALAJUELA,GUACIMA,7,24598.0
4,ASALTO,ARMA BLANCA,2015,ALAJUELA,ALAJUELA,LA GARITA,1,8894.0


In [92]:
# Imprimir datos NA

# police_final[police_final["Poblacion"].isna()].to_excel('prueba.xlsx')

In [94]:
police_final.dropna(inplace = True)
police_final.isna().sum()

Delito       0
Subdelito    0
Ano          0
Provincia    0
Canton       0
Distrito     0
Count        0
Poblacion    0
dtype: int64